In [1]:
import numpy as np
import scipy.sparse.linalg
import sys, os
sys.path.append(os.path.expanduser('~') + '/repos/pyQSYS/build/qsys')
notebook_dir = os.path.dirname(os.path.abspath('test.py'))
common_lib_dir = os.path.join(notebook_dir, '../common_lib')
sys.path.append(common_lib_dir)
from pyQSYS import *
from pyQSYS import RealSpaceFermions, Matrix
np.set_printoptions(suppress=True)
from numpy import sin, cos, sqrt, exp, conj, pi
import re
from systems import generate_Ops_qwz_spin_polarized

In [2]:
def test_qwz_spin_projected_energy(cluster_name, U = 1, t = 1.0, m = 1.0, N_fill = -1, pbc = False, bias = 0):
    freq_step = 0.01
    freq_max = 10
    system = "qwz"

    print(f"Using {cluster_name} with {system} and {"periodic" if pbc else "open"} boundary condition", flush = True) 
    bc = "pbc" if pbc else "obc"

    if cluster_name[0] == "s":
        pattern = re.compile(r"square_(\d+)")
    else:
        pattern = re.compile(r"hexagonal_(\d+)")
    match1 = pattern.search(cluster_name)
    number1 = match1.group(1) if match1 else None
    cluster_size = int(number1)
    model = RealSpaceFermions(clusterName = cluster_name, Spin = 1, Orbitals = 2)
    if N_fill == -1:
        N_fill = model.cluster_size
        
    statemap = model.createStateMap(N = N_fill)
    print(f"Filling is is N = {N_fill}")
    #statemap = model.createStateMap()

    positions = np.array([model.sites[i].r for i in range(cluster_size)])

    print(f"Evaluating the spectrum information for U = {U}, t = {t}, m = {m}.", flush=True)

    H_Op, J_mu_Op = generate_Ops_qwz_spin_polarized(model, cluster_size, t = t, m = m, U = U, pbc = pbc)
    H_Op += bias * nD_(model.sites[0], model.orbitals[0])
    Ham = Matrix('cdouble', H_Op, model, statemap)

    evals, evecs = scipy.sparse.linalg.eigsh(Ham, k = 5, which = 'SA')
    sorted_evals = np.sort(evals)
    sorted_evecs = evecs[:, np.argsort(evals)]
    evals, evecs = sorted_evals, sorted_evecs
    #evals, evecs = np.linalg.eigh(Ham.todense())

    print(f"The low energy spectrum consists of energies: {evals[:5]}")

In [3]:
cluster_name = "square_4d0"
#cluster_name = "square_9t0"
U = 1
t = 1.0
m = 5.0
pbc = True
N_fill = 2
bias = 0.1
test_qwz_spin_projected_energy(cluster_name, U = U, t = t, m = m, pbc = pbc, N_fill = N_fill, bias = bias)


Using square_4d0 with qwz and periodic boundary condition
Filling is is N = 2
Evaluating the spectrum information for U = 1, t = 1.0, m = 5.0.
The low energy spectrum consists of energies: [-13.97538672 -13.92539454  -9.95000781  -9.94999219  -7.74395377]
